# Prepare environment

In [1]:
!pip install git+https://github.com/katarinagresova/ensembl_scraper.git

  Cloning https://github.com/katarinagresova/ensembl_scraper.git to /private/var/folders/q7/pcbd1fnn22l233m3cmvvx1nm0000gn/T/pip-req-build-0px4w0y7
  Running command git clone --filter=blob:none -q https://github.com/katarinagresova/ensembl_scraper.git /private/var/folders/q7/pcbd1fnn22l233m3cmvvx1nm0000gn/T/pip-req-build-0px4w0y7
  Resolved https://github.com/katarinagresova/ensembl_scraper.git to commit 8239db77fd3f0374300cd4d1efac319fb28f3535
  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for scraper, since package 'wheel' is not installed.
  Attempting uninstall: scraper
    Found existing installation: scraper 0.0.1
    Uninstalling scraper-0.0.1:
      Successfully uninstalled scraper-0.0.1
    Running setup.py install for scraper ... done


In [2]:
!wget https://raw.githubusercontent.com/katarinagresova/ensembl_scraper/main/requirements.txt

--2021-10-18 15:16:42--  https://raw.githubusercontent.com/katarinagresova/ensembl_scraper/main/requirements.txt
Prevádza sa raw.githubusercontent.com (raw.githubusercontent.com) na IP adresu... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Pripájanie k raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... pripojené.
HTTP požiadavka odoslaná, čakám na odpoveď... 200 OK
Dĺžka:  337 [text/plain]
Ukladá sa do: `requirements.txt'

requirements.txt    100%[===================>]     337  --.-KB/s    za 0s      

2021-10-18 15:16:43 (10,0 MB/s) - `requirements.txt' uložené [337/337]



In [3]:
!pip install -r requirements.txt

# Create config file

In [4]:
import yaml

config = {
    "root_dir": "../../datasets/",
    "organisms": {
        "homo_sapiens": {
            "external_feature"
        }
    }
}

user_config = 'user_config.yaml'
with open(user_config, 'w') as handle:
  yaml.dump(config, handle)

# Prepare directories

In [5]:
from pathlib import Path

BASE_FILE_PATH = Path("../../datasets/human_enhancers_ensembl/")

# copied from https://stackoverflow.com/a/57892171
def rm_tree(pth: Path):
    for child in pth.iterdir():
        if child.is_file():
            child.unlink()
        else:
            rm_tree(child)
    pth.rmdir()

if BASE_FILE_PATH.exists():
    rm_tree(BASE_FILE_PATH)

# Run tool

In [ ]:
!python -m scraper.ensembl_scraper -c user_config.yaml

# Cleaning

In [39]:
!mv ../../datasets/homo_sapiens_external_feature_enhancer ../../datasets/human_enhancers_ensembl/

In [ ]:
!rm requirements.txt user_config.yaml

# Final reformating

  * gzip all CSV files
  * add extra formatting to yaml config file

In [44]:
!find ../../datasets/human_enhancers_ensembl/ -type f -name "*.csv" -exec gzip {} \;

In [ ]:
with open("../../datasets/human_enhancers_ensembl/metadata.yaml", "r") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

config

In [ ]:
config['classes']['positive']['extra_processing'] = 'ENSEMBL_HUMAN_GENOME' 
config['classes']['negative']['extra_processing'] = 'ENSEMBL_HUMAN_GENOME' 

config

In [43]:
with open("../../datasets/human_enhancers_ensembl/metadata.yaml", 'w') as handle:
  yaml.dump(config, handle)